
# Introduction to Machine Learning

An interactive Machine Learning example written in Python, broken into the following sections:

1. [Scope](#scope)
  1. [Problem Definition](#scope-problem-definition)
  1. [Data](#scope-data)
1. [Prepare](#prepare)
  1. [Import](#prepare-import)
  1. [Train Test Split](#prepare-train-test-split)
  1. [Cleaning](#prepare-cleaning)
1. [Analyze](#analyze)
  1. [Pre-processing](#analyze-pre-processing)
  1. [Exploratory Data Analysis](#analyze-exploratory-data-analysis)
  1. [Modeling](#analyze-modeling)
  1. [Hyper-parameter Tuning](#analyze-hyper-parameter-tuning)
1. [Reflect](#reflect)

<a id='scope'></a>

## Scope

<a id='scope-problem-definition'></a>

### Problem Definition

We seek to answer the following questions by applying Machine Learning techniques to the NYC Mayor's Office of Sustainability dataset on green buildings and energy efficiency:

1. Given a buidling's characteristics, can we predict what its energy usage intensity will be?
  - Could be useful for urban planners to proactively estimate energy consumption of various of living spaces.
1. Given a buidling's characteristics and energy use, can we predict what its Energy Star rating would be?
  - Could be useful for Energy Star to assist in assigning ratings to buildings without a score.
  - Could be useful for architects to predict the Energy Star scores of different designs.
1. What sources of energy cause the most greenhouse gas emissions for a property?
  - Could be useful for policy makers to see what sources of energy produce the most greenhouse gas emissions.
1. What factors lead to energy loss between the source of generation and the site of the property?
  - Could be useful for engineers to identify which locations, property usages, sources of energy, etc. lead to the most energy loss and adjust the energy grid accordingly.

<a id='scope-data'></a>

### Data

#### Citation

```
NYC Mayor's Office of Sustainability, Green Buildings & Energy Efficiency. (2017). [Data set]. 
    Retrieved from http://www.nyc.gov/html/gbee/html/plan/ll84_scores.shtml
```

#### Dimensions

| Column                                                     | Functional Data Type    | Technical Data Type |
|------------------------------------------------------------|-------------------------|---------------------|
| Order                                                      | index                   | integer             |
| Property Id                                                | identifier              | text                |
| Property Name                                              | identifier              | text                |
| Parent Property Id                                         | identifier              | text                |
| Parent Property Name                                       | identifier              | text                |
| BBL - 10 digits                                            | identifier              | text                |
| NYC Borough, Block and Lot (BBL) self-reported             | identifier              | text                |
| NYC Building Identification Number (BIN)                   | identifier              | text                |
| Address 1 (self-reported)                                  | location                | text                |
| Address 2                                                  | location                | text                |
| Postal Code                                                | location                | text                |
| Street Number                                              | location                | text                |
| Street Name                                                | location                | text                |
| Borough                                                    | location                | catagorical         |
| DOF Gross Floor Area                                       | building characteristic | numeric             |
| Primary Property Type - Self Selected                      | building characteristic | categorical         |
| List of All Property Use Types at Property                 | building characteristic | [categorical]       |
| Largest Property Use Type                                  | building characteristic | categorical         |
| Largest Property Use Type - Gross Floor Area (ft²)         | building characteristic | numeric             |
| 2nd Largest Property Use Type                              | building characteristic | categorical         |
| 2nd Largest Property Use - Gross Floor Area (ft²)          | building characteristic | numeric             |
| 3rd Largest Property Use Type                              | building characteristic | categorical         |
| 3rd Largest Property Use Type - Gross Floor Area (ft²)     | building characteristic | numeric             |
| Year Built                                                 | building characteristic | time                |
| Number of Buildings - Self-reported                        | building characteristic | integer             |
| Occupancy                                                  | building characteristic | percentage          |
| Metered Areas (Energy)                                     | building characteristic | categorical         |
| Metered Areas (Water)                                      | building characteristic | categorical         |
| ENERGY STAR Score                                          | score                   | numeric             |
| Site EUI (kBtu/ft²)                                        | energy usage            | numeric             |
| Weather Normalized Site EUI (kBtu/ft²)                     | energy usage            | numeric             |
| Weather Normalized Site Electricity Intensity (kWh/ft²)    | energy usage            | numeric             |
| Weather Normalized Site Natural Gas Intensity (therms/ft²) | energy usage            | numeric             |
| Weather Normalized Source EUI (kBtu/ft²)                   | energy usage            | numeric             |
| Fuel Oil #1 Use (kBtu)                                     | energy usage            | numeric             |
| Fuel Oil #2 Use (kBtu)                                     | energy usage            | numeric             |
| Fuel Oil #4 Use (kBtu)                                     | energy usage            | numeric             |
| Fuel Oil #5 &amp; 6 Use (kBtu)                             | energy usage            | numeric             |
| Diesel #2 Use (kBtu)                                       | energy usage            | numeric             |
| District Steam Use (kBtu)                                  | energy usage            | numeric             |
| Natural Gas Use (kBtu)                                     | energy usage            | numeric             |
| Weather Normalized Site Natural Gas Use (therms)           | energy usage            | numeric             |
| Electricity Use - Grid Purchase (kBtu)                     | energy usage            | numeric             |
| Weather Normalized Site Electricity (kWh)                  | energy usage            | numeric             |
| Total GHG Emissions (Metric Tons CO2e)                     | environmental footprint | numeric             |
| Direct GHG Emissions (Metric Tons CO2e)                    | environmental footprint | numeric             |
| Indirect GHG Emissions (Metric Tons CO2e)                  | environmental footprint | numeric             |
| Property GFA - Self-Reported (ft²)                         | building characteristic | numeric             |
| Water Use (All Water Sources) (kgal)                       | resource usage          | numeric             |
| Water Intensity (All Water Sources) (gal/ft²)              | resource usage          | numeric             |
| Source EUI (kBtu/ft²)                                      | energy usage            | numeric             |
| Release Date                                               | metadata                | time                |
| Water Required?                                            | building characteristic | categorical         |
| DOF Benchmarking Submission Status                         | metadata                | categorical         |

<a id='prepare'></a>

## Prepare

<a id='prepare-import'></a>

### Import

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import the data
data_path = os.path.join(os.path.abspath('..'), 'data', 'nyc_benchmarking_disclosure_data_reported_in_2017.xlsx')
df = pd.read_excel(data_path, sheet_name='Information and Metrics')

In [3]:
# View the structure of the data
df.head()

,Order,Property Id,Property Name,Parent Property Id,Parent Property Name,BBL - 10 digits,"NYC Borough, Block and Lot (BBL) self-reported",NYC Building Identification Number (BIN),Address 1 (self-reported),Address 2,...,Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Property GFA - Self-Reported (ft²),Water Use (All Water Sources) (kgal),Water Intensity (All Water Sources) (gal/ft²),Source EUI (kBtu/ft²),Release Date,Water Required?,DOF Benchmarking Submission Status
0,1,13286,201/205,13286,201/205,1013160001,1013160001,1037549,201/205 East 42nd st.,Not Available,...,6962.2,0,6962.2,762051,Not Available,Not Available,619.4,2017-05-01 17:32:03,No,In Compliance
1,2,28400,NYP Columbia (West Campus),28400,NYP Columbia (West Campus),1021380040,1-02138-0040,1084198; 1084387;1084385; 1084386; 1084388; 10...,622 168th Street,Not Available,...,55870.4,51016.4,4854.1,3889181,Not Available,Not Available,404.3,2017-04-27 11:23:27,No,In Compliance
2,3,4778226,MSCHoNY North,28400,NYP Columbia (West Campus),1021380030,1-02138-0030,1063380,3975 Broadway,Not Available,...,0,0,0,231342,Not Available,Not Available,Not Available,2017-04-27 11:23:27,No,In Compliance
3,4,4778267,Herbert Irving Pavilion & Millstein Hospital,28400,NYP Columbia (West Campus),1021390001,1-02139-0001,1087281; 1076746,161 Fort Washington Ave,177 Fort Washington Ave,...,0,0,0,1305748,Not Available,Not Available,Not Available,2017-04-27 11:23:27,No,In Compliance
4,5,4778288,Neuro Institute,28400,NYP Columbia (West Campus),1021390085,1-02139-0085,1063403,710 West 168th Street,Not Available,...,0,0,0,179694,Not Available,Not Available,Not Available,2017-04-27 11:23:27,No,In Compliance


<a id='prepare-train-test-split'></a>

### Train Test Split

In [ ]:
# Hold out a portion of the data for testing
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.25)

# Furthermore, hold out a portion of the training set for validation
df_train, df_valid = train_test_split(df_train, test_size=0.25)

# Print the shape of each
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

<a id='prepare-cleaning'></a>

### Cleaning

#### Column Types

In [ ]:
# We will define each cleaning step as a function,
# so that we can apply the same rules to our validation and test sets
def clean_missing_value_text(df, text):
    return df.replace(text, np.NaN)

In [ ]:
def clean_col_types(df):
    return df.astype(dtype={'Order': 'int64',
                            'Property Id': 'object',
                            'Property Name': 'object',
                            'Parent Property Id': 'object',
                            'Parent Property Name': 'object',
                            'BBL - 10 digits': 'object',
                            'NYC Borough, Block and Lot (BBL) self-reported': 'object',
                            'NYC Building Identification Number (BIN)': 'object',
                            'Address 1 (self-reported)': 'object',
                            'Address 2': 'object',
                            'Postal Code': 'object',
                            'Street Number': 'object',
                            'Street Name': 'object',
                            'Borough': 'category',
                            'DOF Gross Floor Area': 'float64',
                            'Primary Property Type - Self Selected': 'category',
                            'List of All Property Use Types at Property': 'object',
                            'Largest Property Use Type': 'category',
                            'Largest Property Use Type - Gross Floor Area (ft²)': 'float64',
                            '2nd Largest Property Use Type': 'category',
                            '2nd Largest Property Use - Gross Floor Area (ft²)': 'float64',
                            '3rd Largest Property Use Type': 'category',
                            '3rd Largest Property Use Type - Gross Floor Area (ft²)': 'float64',
                            'Year Built': 'int64',
                            'Number of Buildings - Self-reported': 'int64',
                            'Occupancy': 'float64',
                            'Metered Areas (Energy)': 'category',
                            'Metered Areas  (Water)': 'category',
                            'ENERGY STAR Score': 'float64',
                            'Site EUI (kBtu/ft²)': 'float64',
                            'Weather Normalized Site EUI (kBtu/ft²)': 'float64',
                            'Weather Normalized Site Electricity Intensity (kWh/ft²)': 'float64',
                            'Weather Normalized Site Natural Gas Intensity (therms/ft²)': 'float64',
                            'Weather Normalized Source EUI (kBtu/ft²)': 'float64',
                            'Fuel Oil #1 Use (kBtu)': 'float64',
                            'Fuel Oil #2 Use (kBtu)': 'float64',
                            'Fuel Oil #4 Use (kBtu)': 'float64',
                            'Fuel Oil #5 & 6 Use (kBtu)': 'float64',
                            'Diesel #2 Use (kBtu)': 'float64',
                            'District Steam Use (kBtu)': 'float64',
                            'Natural Gas Use (kBtu)': 'float64',
                            'Weather Normalized Site Natural Gas Use (therms)': 'float64',
                            'Electricity Use - Grid Purchase (kBtu)': 'float64',
                            'Weather Normalized Site Electricity (kWh)': 'float64',
                            'Total GHG Emissions (Metric Tons CO2e)': 'float64',
                            'Direct GHG Emissions (Metric Tons CO2e)': 'float64',
                            'Indirect GHG Emissions (Metric Tons CO2e)': 'float64',
                            'Property GFA - Self-Reported (ft²)': 'float64',
                            'Water Use (All Water Sources) (kgal)': 'float64',
                            'Water Intensity (All Water Sources) (gal/ft²)': 'float64',
                            'Source EUI (kBtu/ft²)': 'float64',
                            'Release Date': 'int64',
                            'Water Required?': 'category',
                            'DOF Benchmarking Submission Status': 'category'})

In [ ]:
# Notice above that missing values are actually listed as 'Not Available'
# We cannot set columns to a numeric data type correctly when they contain the 'Not Available' text
# Instead, we can replace each 'Not Available' with NaN, which is widely recognized by Python libraries
df_train = clean_missing_value_text(df_train, text='Not Available')

# See what the column types are now
# Will want to change these based on the dimensions table above
print()
print('Data types - before cleaning')
print('----------------------------')
df_train.info()

# Clean the column types
df_train = clean_col_types(df_train)

# Confirm that the types were changed
print()
print('Data types - after cleaning')
print('---------------------------')
df_train.info()

#### Missing Values

In [ ]:
# Function to calculate the percent of missing values for each column in a data frame
def calc_percent_nan_col(df):
    return df.isna().sum(axis=0)/df.shape[0] * 100

In [ ]:
# Function to calculate the percent of missing values for each row in a data frame
def calc_percent_nan_row(df):
    return df.isna().sum(axis=1)/df.shape[1] * 100

In [ ]:
# Function to pretty print pandas objects in percent format (like 100.00%)
def pprint_percent(pd_obj):
    
    # Format percents like 100.00%
    original_format = pd.options.display.float_format
    pd.options.display.float_format = '{:.2f}%'.format
    
    # Print
    print()
    print(pd_obj)
    print()
    
    # Reset the pandas formatter
    pd.options.display.float_format = original_format

In [ ]:
# Function to remove columns and rows that have over a certain percent missing values
def clean_missing_values(df, col_perc=33, row_perc=33):

    # Remove all columns that have more than 33% missing values
    df = df.loc[:, calc_percent_nan_col(df) < col_perc]
    
    # Remove all rows that have more than 33% missing values
    df = df.loc[calc_percent_nan_row(df) < row_perc, :]
        
    return df

In [ ]:
# See what percent of each column in the training set is missing values
print()
print('Percent NaNs in each column - before cleaning')
print('---------------------------------------------')
pprint_percent(calc_percent_nan_col(df_train))

# See what percent of each row in the training set is missing values
hist = calc_percent_nan_row(df_train).hist()
print()
print('Percent NaNs in each row - before cleaning')
print('------------------------------------------')
plt.show()

# Clean
df_train = clean_missing_values(df_train)

# For just the training set, also remove rows missing the target
df_train = df_train.loc[-df_train.loc[:, 'ENERGY STAR Score'].isna(), :]

# See what percent of each column in the training set is missing values, after cleaning
print()
print('Percent NaNs in each column - after cleaning')
print('--------------------------------------------')
pprint_percent(calc_percent_nan_col(df_train))

# See what percent of each row in the training set is missing values, after cleaning
print()
print('Percent NaNs in each row - after cleaning')
print('-----------------------------------------')
hist = calc_percent_nan_row(df_train).hist()
plt.show()


#### Outliers

In [ ]:
# https://www.itl.nist.gov/div898/handbook/prc/section1/prc16.htm
def clean_outliers(df, lower_quantile=0.25, upper_quantile=0.75, lower_multiplier=3, upper_multiplier=3):
    
    # Inner quartile
    q1 = df.quantile(lower_quantile)
    q3 = df.quantile(upper_quantile)
    iq = q3 - q1

    # Lower bound
    lb = q1 - lower_multiplier * iq

    # Upper bound
    ub = q3 + upper_multiplier * iq

    # Remove the outliers (but keep missing values)
    for col in lb.index:
        df = df.loc[(df[col] >= lb[col]) | df[col].isna(), :] # Remove lower-bound outliers
        df = df.loc[(df[col] <= ub[col]) | df[col].isna(), :] # Remove upper-bound outliers
    return df

In [ ]:
# Check the distributions of numeric columns before cleaning outliers
# Note the large domain
print()
print('Histograms - before cleaning')
print('----------------------------')
hist = df_train.hist(figsize=(30, 30))
plt.show()

# Remove outliers
df_train = clean_outliers(df_train)

# Now we have cleaner numeric columns
print()
print('Histograms - after cleaning')
print('---------------------------')
hist = df_train.hist(figsize=(30, 30))
plt.show()

<a id='analyze'></a>

## Analyze

<a id='analyze-pre-processing'></a>

### Pre-processing
Imputation (move from above)
Normalization (both 0-1 and normal dist)
Feature engineering
Feature selection
Colinear features (PCA)


#### Imputation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)

In [ ]:
def pre_process_mean_median_impute(df, remove=False, flag=False):
    
    # Flag if desired
    if flag:
        df['imputed'] = 0
        df = df.astype({'imputed': 'int64'})

    # Remove rows that cannot be imputed
    if remove:
        df = df.loc[-df.select_dtypes(include=['object']).isna().any(axis=1), :]
        
    # It does not make sense to impute plain text (object) types,
    # as plain text does not have a finite number of instances
    perc_nan_col = calc_percent_nan_col(df.select_dtypes(exclude=['object']))

    # For each column that has missing values, impute them using median or mode
    for col_name in perc_nan_col[perc_nan_col > 0].index:
        
        # Get the data type of the column
        dtype = df.dtypes[col_name]
        
        # Get the column itself
        col = df.loc[:, col_name]
                
        # Get the y column itself
        col = df.loc[:, col_name]
        col_isna = col.isna()
        
        # If numeric, use the median
        if dtype == 'int64' or dtype == 'float64':
            col.fillna(col.median(), inplace=True)
            
        # Else, use the mode
        else:
            col.fillna(col.mode().iloc[0], inplace=True)
                
        # Flag if desired
        if flag:
            df.loc[col_isna, 'imputed'] = 1
    
    # Return the df
    return df

In [ ]:
def pre_process_knn_impute(df, n_neighbors=5, weights='uniform', remove=False, flag=False):
    
    # Flag if desired
    if flag:
        df['imputed'] = 0
        df = df.astype({'imputed': 'int64'})

    # Remove rows that cannot be imputed
    if remove:
        df = df.loc[-df.select_dtypes(include=['object']).isna().any(axis=1), :]
    
    # The classifier
    knn_classifier = KNeighborsClassifier(n_neighbors, weights=weights)
    
    # For each column that has missing values,
    # Train a KNN model to predict those missing values based on the data that is not missing
    perc_nan_col = calc_percent_nan_col(df)
    for col_name_y in perc_nan_col[perc_nan_col > 0].index:
                        
        # Get the y column itself
        print('Imputing ' + col_name_y + '...')
        col_y = df.loc[:, col_name_y]
        col_y_isna = col_y.isna()
        
        # Unable to impute object types
        if df.dtypes[col_name_y] == 'object':
            continue
        
        # For referencing X columns
        col_name_X = df.columns != col_name_y
     
        # Train the model using data that is not missing
        
        # Prepare X
        knn_X = df.loc[:, col_name_X].select_dtypes(exclude='object') # Don't include objects
        knn_X = pre_process_mean_median_impute(knn_X) # Do some preliminary imputation
        knn_X = pd.get_dummies(knn_X) # Convert categorical columns to dummy numeric vars
        knn_train_X = knn_X.loc[-col_y_isna, :]
        knn_impute_X = knn_X.loc[col_y_isna, :]
        
        # Prepare y
        knn_train_y = df.loc[-col_y_isna, col_name_y]
        le_y = LabelEncoder().fit(knn_train_y)
        knn_train_y = le_y.transform(knn_train_y) # Must be transformed to a category
        
        # Fit X -> y
        knn_classifier.fit(knn_train_X, knn_train_y)
        
        # Impute missing y <- X
        knn_impute_y = knn_classifier.predict(knn_impute_X)
        knn_impute_y = le_y.inverse_transform(knn_impute_y) # Must be inverse-transformed back
        df.loc[col_y_isna, col_name_y] = knn_impute_y
        
        # Flag if desired
        if flag:
            df.loc[col_y_isna, 'imputed'] = 1

    # Return the df
    return df

In [ ]:
# Percent missing values before imputation
print()
print('Percent NaNs in each column - before pre-processing')
print('---------------------------------------------------')
pprint_percent(calc_percent_nan_col(df_train))

# Impute using median (for numeric columns) and mode (for all others)
df_train = pre_process_knn_impute(df_train, remove=True, flag=True)
#df_train = pre_process_mean_median_impute(df_train, remove=True, flag=True)

# Percent missing values after imputation
print()
print('Percent NaNs in each column - after pre-processing')
print('--------------------------------------------------')
pprint_percent(calc_percent_nan_col(df_train))

<a id='analyze-exploratory-data-analysis'></a>

### Exploratory Data Analysis

#### Colinearity and Pair Plots

In [ ]:
# Seaborn visualization library
import seaborn as sns
warnings.filterwarnings(module='scipy*', action='ignore', category=FutureWarning)

# Floor-area related features
g = sns.pairplot(df_train.sample(1000), 
                 vars=['DOF Gross Floor Area',
                       'Largest Property Use Type - Gross Floor Area (ft²)',
                       'Property GFA - Self-Reported (ft²)'], height=3, hue='imputed')

In [ ]:
# EUI-related features
g = sns.pairplot(df_train.sample(1000), 
                 vars=['Site EUI (kBtu/ft²)',
                       'Source EUI (kBtu/ft²)',
                       'Weather Normalized Site EUI (kBtu/ft²)',
                       'Weather Normalized Source EUI (kBtu/ft²)',
                       'Weather Normalized Site Electricity Intensity (kWh/ft²)',
                       'Weather Normalized Site Natural Gas Intensity (therms/ft²)'], height=4, hue='imputed')

<a id='analyze-modeling'></a>

### Modeling

<a id='analyze-hyper-parameter-tuning'></a>

### Hyper-parameter Tuning

<a id='reflect'></a>

## Reflect